# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-tf-idf-circuit-court-data" data-toc-modified-id="Generate-tf-idf-circuit-court-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate tf-idf circuit court data</a></div><div class="lev2 toc-item"><a href="#Save-output" data-toc-modified-id="Save-output-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Save output</a></div><div class="lev3 toc-item"><a href="#Save-sparse-matrix-representation" data-toc-modified-id="Save-sparse-matrix-representation-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Save sparse matrix representation</a></div><div class="lev1 toc-item"><a href="#Case-similarity-approach" data-toc-modified-id="Case-similarity-approach-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Case similarity approach</a></div><div class="lev3 toc-item"><a href="#Algorithms" data-toc-modified-id="Algorithms-201"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Algorithms</a></div><div class="lev3 toc-item"><a href="#Steps" data-toc-modified-id="Steps-202"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Steps</a></div><div class="lev3 toc-item"><a href="#Evaluation" data-toc-modified-id="Evaluation-203"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Evaluation</a></div><div class="lev3 toc-item"><a href="#Tf-idf-model-current-accuracy" data-toc-modified-id="Tf-idf-model-current-accuracy-204"><span class="toc-item-num">2.0.4&nbsp;&nbsp;</span>Tf-idf model current accuracy</a></div><div class="lev4 toc-item"><a href="#Cosine-similarity-model-current-accuracy" data-toc-modified-id="Cosine-similarity-model-current-accuracy-2041"><span class="toc-item-num">2.0.4.1&nbsp;&nbsp;</span>Cosine similarity model current accuracy</a></div>

In [2]:
import pandas as pd
import BeautifulSoup as bs
import os
import graphlab as gl
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from scipy import io
import numpy as np
import math
from numpy import inf

# Generate tf-idf circuit court data

In [2]:
data = gl.SFrame({'filename':[""], 'text':[""]})
for root, dirs, files in os.walk('/Users/shiv/Desktop/circuit-scbd-mapped-files/', topdown=False):
    for idx,name in enumerate(files):
        if "html" in name:
            soup = bs.BeautifulSoup(open(os.path.join(root, name)))
            for elem in soup.findAll(['script', 'style']):
                elem.extract()
            res = soup.find("div", {"id": "content_document"})
            df = gl.SFrame({'filename':[name], 'text':[str(res.text.encode('utf-8'))]})
            data = data.append(df)
        print str(idx) + '->' + str(len(files))
docs = gl.text_analytics.count_words(data['text'])
docs = docs.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)
data['tf_idf'] = gl.text_analytics.tf_idf(docs)

This non-commercial license of GraphLab Create for academic use is assigned to sp4658@nyu.edu and will expire on March 06, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1491774614.log


0->1072
1->1072
2->1072
3->1072
4->1072
5->1072
6->1072
7->1072
8->1072
9->1072
10->1072
11->1072
12->1072
13->1072
14->1072
15->1072
16->1072
17->1072
18->1072
19->1072
20->1072
21->1072
22->1072
23->1072
24->1072
25->1072
26->1072
27->1072
28->1072
29->1072
30->1072
31->1072
32->1072
33->1072
34->1072
35->1072
36->1072
37->1072
38->1072
39->1072
40->1072
41->1072
42->1072
43->1072
44->1072
45->1072
46->1072
47->1072
48->1072
49->1072
50->1072
51->1072
52->1072
53->1072
54->1072
55->1072
56->1072
57->1072
58->1072
59->1072
60->1072
61->1072
62->1072
63->1072
64->1072
65->1072
66->1072
67->1072
68->1072
69->1072
70->1072
71->1072
72->1072
73->1072
74->1072
75->1072
76->1072
77->1072
78->1072
79->1072
80->1072
81->1072
82->1072
83->1072
84->1072
85->1072
86->1072
87->1072
88->1072
89->1072
90->1072
91->1072
92->1072
93->1072
94->1072
95->1072
96->1072
97->1072
98->1072
99->1072
100->1072
101->1072
102->1072
103->1072
104->1072
105->1072
106->1072
107->1072
108->1072
109->1072
110->1072


In [4]:
data.save('/Users/shiv/Desktop/data')

## Save output

In [29]:
data = gl.load_sframe('/Users/shiv/Desktop/data')
data = data[1:]
data = data.add_row_number()

In [23]:
def dataframe_to_scipy(x, column_name):
    '''
    Convert a dictionary column of an SFrame into a sparse matrix format where
    each (row_id, column_id, value) triple corresponds to the value of
    x[row_id][column_id], where column_id is a key in the dictionary.
       
    Example
    >>> sparse_matrix, map_key_to_index = sframe_to_scipy(sframe, column_name)
    '''
    assert x[column_name].dtype() == dict, \
        'The chosen column must be dict type, representing sparse data.'
        
    # Create triples of (row_id, feature_id, count).
    # 1. Stack will transform x to have a row for each unique (row, key) pair.
    x = x.stack(column_name, ['feature', 'value'])

    # Map words into integers using a OneHotEncoder feature transformation.
    f = gl.feature_engineering.OneHotEncoder(features=['feature'])
    # 1. Fit the transformer using the above data.
    f.fit(x)
    # 2. The transform takes 'feature' column and adds a new column 'feature_encoding'.
    x = f.transform(x)
    # 3. Get the feature mapping.
    mapping = f['feature_encoding']
    # 4. Get the feature id to use for each key.
    x['feature_id'] = x['encoded_features'].dict_keys().apply(lambda x: x[0])

    # Create numpy arrays that contain the data for the sparse matrix.
    i = np.array(x['id'])
    j = np.array(x['feature_id'])
    v = np.array(x['value'])
    
    width = x['id'].max() + 1
    height = x['feature_id'].max() + 1
    # Create a sparse matrix.
    mat = csr_matrix((v, (i, j)), shape=(width, height))

    return mat, mapping

In [30]:
tf_idf, map_index_to_word = dataframe_to_scipy(data, 'tf_idf')

In [14]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

### Save sparse matrix representation 

In [31]:
io.mmwrite("/Users/shiv/Desktop/tf-idf-sparse-merged", tf_idf)

In [5]:
# back to sparse matrix
newm = io.mmread("/Users/shiv/Desktop/tf-idf-sparse.mtx")
tf_idf = newm.tocsr()  

In [101]:
dense = tf_idf.todense()

In [33]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(dense) 

NameError: name 'dense' is not defined

In [17]:
neighs = neigh.kneighbors(tf_idf[128], 10, return_distance=False)

In [18]:
neighs = neighs[0]

In [20]:
similar_cases = data.filter_by(neighs, 'id')['filename']

In [21]:
similar_cases

dtype: str
Rows: 10
['X1HVA7GN.html', 'X3D675.html', 'X3RL8T.html', 'X3SM74.html', 'X3SV1G.html', 'X3SV48.html', 'X3TQKN.html', 'X41ESC.html', 'X42DJD.html', 'XFAPG2.html']

In [34]:
joined = pd.read_csv('/Users/shiv/Desktop/sc_lc.csv')

# Case similarity approach
### Algorithms
1. Using tf-idf and 10 nearest neighbour search
2. Using cosine similarity

### Steps
1. Genearate tf-idf/word vector data for circuit court bloomberg text.
2. Map circuit court data to scbd data ie., use only those circuit court texts which were appealed in supreme court.
3. Train model based on Nearest neighbour model.
4. Predict for all cases.
5. Evaluate outcome

### Evaluation
1. Predict for all scbd cases
2. Since cases are justice centred, each docket will appear 8-9 times (ie., number of judges). Since *case_outcome_disposition* is same for all, use any of them.
3. Take majority vote of 10 nearest neighbour, and take that as predicted output.
4. accuracy = number of correct predictoins / total number of cases

In [6]:
def print_case_outcomes_for_file(file):
    if len(joined[joined['caseid']==file[:-5]]['case_outcome_disposition'].values) > 0:
        return joined[joined['caseid']==file[:-5]]['case_outcome_disposition'].values[0]
    else:
        print "File not found "+file
        return -1

In [7]:
def get_compare_case_outcomes(cases):
    d = []
    affirm = 0
    reverse = 0
    outcome = 0
    for idx,case in enumerate(cases):
        case_outcome = print_case_outcomes_for_file(case)
        if case_outcome == -1:
            outcome = -1
            break
        d.append({'file': case, 'outcome': case_outcome})
        if case_outcome == 1:
            affirm = affirm + 1
        else:
            reverse = reverse + 1
    
    if outcome!=-1:
        if affirm>reverse:
            outcome = 1
        else:
            outcome = 0
    return d,outcome

In [8]:
def get_overall_score_tf_idf():
    correct = 0
    incorrect = 0
    num = tf_idf.shape[0]
    nearest_neighbour_data = {}
    for idx in range(1,num):
        print idx
        neighs = neigh.kneighbors(tf_idf[idx], 10, return_distance=False)[0]
        similar_cases = data.filter_by(neighs, 'id')['filename']
        df,outcome = get_compare_case_outcomes(similar_cases)
        if outcome == -1:
            continue
        query_file = data[data['id']==idx]['filename'] [0]
        actual_outcome = print_case_outcomes_for_file(query_file)
        
        if actual_outcome == outcome:
            correct = correct + 1
        else:
            incorrect = incorrect + 1
        
        nearest_neighbour_data[idx] = {'query_file': query_file, 
                                       'similar_cases': similar_cases, 
                                       'similar_case_outcomes' : df,
                                       'correct':correct,
                                       'incorrect':incorrect
                                      }

    return correct, incorrect, nearest_neighbour_data

In [35]:
def get_overall_score_cosine_similarity():
    def getKey(item):
        return item[0]

    correct = 0
    incorrect = 0
    num = tf_idf.shape[0]
    nearest_neighbour_data = {}
    for idx in range(1,num):
        similarity = cosine_similarity(tf_idf[idx:idx+1], tf_idf)[0]
        indices = range(1,num)
        tuples = zip(similarity,indices)
        tuples = sorted(tuples,reverse=True,key=getKey)
        
        neighs = list()
        for key,val in tuples[0:10]:
            neighs.append(val)
        
        similar_cases = data.filter_by(neighs, 'id')['filename']
        df,outcome = get_compare_case_outcomes(similar_cases)
        if outcome == -1:
            continue
        query_file = data[data['id']==idx]['filename'] [0]
        actual_outcome = print_case_outcomes_for_file(query_file)
        
        if actual_outcome == outcome:
            correct = correct + 1
        else:
            incorrect = incorrect + 1
        
        nearest_neighbour_data[idx] = {'query_file': query_file, 
                                       'similar_cases': similar_cases, 
                                       'similar_case_outcomes' : df,
                                       'correct':correct,
                                       'incorrect':incorrect
                                      }

    return correct, incorrect, nearest_neighbour_data

In [ ]:
# Some files are not found, could not download all files post 1975
overall_correct, overall_incorrect, nearest_neighbour_data = get_overall_score_tf_idf()
accuracy = float(overall_correct)/float(overall_incorrect+overall_correct)

### Tf-idf model current accuracy

In [1]:
accuracy

0.698146812333


In [36]:
# Some files are not found, could not download all files post 1975
overall_correct, overall_incorrect, nearest_neighbour_data = get_overall_score_cosine_similarity()

File not found X4181C.html
File not found X45NHK.html
File not found X45KTC.html
File not found X45NHK.html
File not found X1459TI003.html
File not found X147ATI003.html
File not found X147ATI003.html
File not found X147ATI003.html
File not found X45NHK.html
File not found X45NHK.html
File not found XGFRCE.html
File not found X147ATI003.html
File not found XGFRCE.html
File not found X147ATI003.html
File not found X45NHK.html
File not found X45NHK.html
File not found X147ATI003.html
File not found X45NHK.html
File not found X45NHK.html
File not found X45NHK.html
File not found X147ATI003.html
File not found X45NHK.html
File not found X45NHK.html
File not found X4181C.html
File not found X4181C.html
File not found X41SAO.html
File not found X4181C.html
File not found X4181C.html
File not found X4181C.html
File not found X4181C.html
File not found X3LPSO.html
File not found X45KTC.html
File not found X1459TI003.html
File not found X3DI2F.html
File not found X3DI2F.html
File not found X147

In [89]:
overall_correct, overall_incorrect, nearest_neighbour_data = get_overall_score()

1
File not found X10CUPE003.html
2
3
4
5
6
7
8
9
10
11
12
13
File not found X1459TI003.html
14
File not found X147ATI003.html
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
File not found X10CUPE003.html
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
File not found X3DI2F.html
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
2

In [37]:
overall_correct

597

In [38]:
overall_incorrect

385

#### Cosine similarity model current accuracy

In [39]:
accuracy = float(overall_correct)/float(overall_incorrect+overall_correct)

In [40]:
accuracy

0.6079429735234216

In [41]:
nearest_neighbour_data = pd.DataFrame(nearest_neighbour_data)

In [43]:
nearest_neighbour_data.transpose()

,correct,incorrect,query_file,similar_case_outcomes,similar_cases
3,0,1,X126AOG003.html,"[{u'outcome': 1, u'file': u'X12D52I003.html'},...","(X12D52I003.html, X18SD9E003.html, X1CEE96003...."
6,0,2,X12I4MO003.html,"[{u'outcome': 0, u'file': u'X13P046003.html'},...","(X13P046003.html, X16AKIQ003.html, X3CMNT.html..."
7,0,3,X13P046003.html,"[{u'outcome': 0, u'file': u'X13TEK2003.html'},...","(X13TEK2003.html, X1FE3G0003.html, X1HUG6K003...."
8,0,4,X13TEK2003.html,"[{u'outcome': 1, u'file': u'X13VBM8003.html'},...","(X13VBM8003.html, X15139I003.html, X1690BO003...."
9,1,4,X13VBM8003.html,"[{u'outcome': 1, u'file': u'X141L2A003.html'},...","(X141L2A003.html, X16F774003.html, X1FKIPK003...."
10,2,4,X141L2A003.html,"[{u'outcome': 1, u'file': u'X125RLM003.html'},...","(X125RLM003.html, X12FF34003.html, X141L2O003...."
13,2,5,X147ATI003.html,"[{u'outcome': 1, u'file': u'X148SQ4003.html'},...","(X148SQ4003.html, X16HPN8003.html, X16K8IC003...."
14,3,5,X148SQ4003.html,"[{u'outcome': 0, u'file': u'X14ADJA003.html'},...","(X14ADJA003.html, X18J7U8003.html, X1AGO06003...."
15,3,6,X14ADJA003.html,"[{u'outcome': 1, u'file': u'X14CH0M003.html'},...","(X14CH0M003.html, X18TN5A003.html, X1B6SUE003...."
16,4,6,X14CH0M003.html,"[{u'outcome': 1, u'file': u'X14HLRI003.html'},...","(X14HLRI003.html, X18SD9E003.html, X3G7NG.html..."


In [62]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [59]:
f = pd.read_csv('/Users/shiv/.bin/10_scotus/sc_lc_link.csv')

In [64]:
f

,Unnamed: 0,title,action,caseid,link,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,caseOrigin,caseOriginState,caseSource,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,...,ReversedInPart,Vacated,VacatedInPart,Remanded,Treat,Dissenting1,Dissenting2,JudgesCount,JudgesListTouse,DissentOrConcurenceOpinionOnly,dissentOrconcurCaseid,DissentOrConcuOnlyIncluded,Judgeconcurring,JudgeDissenting,MultipleDissents,PossibleTypoInDissenterOrJudge,RehearingOrPetition,ConcurenceMistakenForDissent,JudgeDissentingTouse1,JudgeDissentingTouse2,Author,Writer,year,Criminal,Civil_Rights,First_Amendment,Due_Process,Privacy,Labor_Relations,Econ_Activity,Miscellanous,unknown_circuit,federal_cir,cir1,cir2,cir3,cir4,cir5,cir6,cir7,cir8,cir9,cir10,cir11,cir12,within,outside,concurdummy,mergeCircuit,Circuitsource
0,0,"Reichle v. Howards, 132 S. Ct. 2088, 182 L. Ed...",reversing the judgment and remanding the case in,X1F3VQS003,True,2011-061-01-01,2011-061-01-01-01,2012-06-04,1,NaN,132 S. Ct. 2088,182 L. Ed. 2d 985,2012 U.S. LEXIS 4132,2011,1704,Roberts,11-262,"REICHLE, JR., et al., PETITIONERS v. STEVEN H...",2012-03-21,27.0,NaN,100.0,NaN,1,NaN,NaN,52.0,NaN,30.0,dissent in lower court,11.0,7.0,2.0,1,4,no unusual disposition,petitioner won,precedent unaltered,vote clearly specified,20400.0,2.0,1,dissent in opp. direction,1.0,NaN,2.0,1st Amend. (speech),NaN,CThomas,JGRoberts,...,1,0,0,1,6.0,1.0,NaN,3,KELLY | LUCERO | SEYMOUR,0,NaN,0.0,KELLY,KELLY,0,0,0,0,KELLY,NaN,SEYMOUR,3.0,2011.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,5.0,1.0,1,matched (3),10.0
1,1,"14 Penn Plaza LLC v. Pyett, 556 U.S. 247, 129 ...",reversing the judgment and remanding the case in,X17CJ3A003,True,2008-041-01-01,2008-041-01-01-01,2009-04-01,1,556 U.S. 247,129 S. Ct. 1456,173 L. Ed. 2d 398,2009 U.S. LEXIS 2497,2008,1702,Roberts,07-581,14 PENN PLAZA LLC et al. v. STEVEN PYETT et al.,2008-12-01,151.0,NaN,247.0,NaN,1,32.0,NaN,96.0,NaN,22.0,no mention of dissent,2.0,2.0,2.0,1,4,no unusual disposition,petitioner won,precedent unaltered,vote clearly specified,20060.0,2.0,1,dissent in opp. direction,4.0,NaN,3.0,Age Discrimination,NaN,CThomas,JGRoberts,...,0,0,0,0,1.0,NaN,NaN,3,CABRANES | KORMAN | RAGGI,0,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,CABRANES,1.0,2007.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,matched (3),2.0
2,2,"149 Madison Ave. Corp. v. Asselta, 331 U.S. 19...",affirming the judgment in,X40PLL,True,1946-096-01-01,1946-096-01-01-01,1947-05-05,1,331 U.S. 199,67 S. Ct. 1178,91 L. Ed. 1432,1947 U.S. LEXIS 2937,1946,1301,Vinson,497,149 MADISON AVENUE CORP. ET AL. v. ASSELTA ET AL.,1947-02-11,151.0,NaN,145.0,NaN,1,NaN,NaN,96.0,37.0,22.0,no mention of dissent,10.0,2.0,2.0,1,2,no unusual disposition,petitioner lost,precedent unaltered,vote clearly specified,70040.0,7.0,2,dissent in opp. direction,4.0,NaN,3.0,334.0,NaN,FMVinson,FMVinson,...,0,0,0,0,1.0,NaN,NaN,3,CHASE | FRANK | CLARK,0,NaN,0.0,NaN,NaN,0,0,0,0,NaN,NaN,PER CURIAM,NaN,1946.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,matched (3),2.0
3,3,"44 Liquormart, Inc. v. Rhode Island, 517 U.S. ...",reversing the judgment in,X3MRHC,True,1995-049-01-01,1995-049-01-01-01,1996-05-13,1,517 U.S. 484,116 S. Ct. 1495,134 L. Ed. 2d 711,1996 U.S. LEXIS 3020,1995,1607,Rehnquist,94-1140,"44 LIQUORMART, INC. AND PEOPLES SUPER LIQUOR S...",1995-11-01,115.0,NaN,28.0,47.0,1,NaN,NaN,113.0,NaN,21.0,no mention of dissent,10.0,3.0,1.0,3,3,no unusual disposition,petitio

In [34]:
soup = bs.BeautifulSoup(open("/Users/shiv/Desktop/2013_complete/X1HVA7GN.html"))
for elem in soup.findAll(['script', 'style']):
    elem.extract()

In [36]:
res = soup.find("div", {"id": "content_document"})

In [37]:
res.text

u'PrimaryFederalReporter,ReporterOfDecisions,Bloomberg,712 F.3d 676106 U.S.P.Q.2d 134112-2786-cv2013 BL 87728Pagination*F.3d**U.S.P.Q.2d***BLMajority Opinion&gt;Dissenting Opinion&gt;United States Court of Appeals for the Second CircuitWNET, Thirteen, Fox Television Stations, Inc., Twentieth Century Fox Film Corporation, WPIX, Inc., Univision Television Group, Inc., The Univision Network Limited Partnership, and Public Broadcasting Service,  Plaintiffs-Counter-Defendants-Appellants,v.Aereo, Inc., f/k/a Bamboom Labs, Inc.,  Defendant-Counter-Claimant-Appellee.American Broadcasting Companies, Inc., Disney Enterprises, Inc., CBS Broadcasting Inc., CBS Studios Inc., NBCUniversal Media, LLC, NBC Studios, LLC, Universal Network Television, LLC, Telemundo Network Group LLC, and WNJU-TV Broadcasting LLC,  Plaintiffs-Counter-Defendants-Appellants,v.Aereo, Inc.,  Defendant-Counter-Claimant-Appellee.Docket Nos.  12-2786-cv,  12-2807-cv.Argued: Nov. 30, 2012.Decided: April 1, 2013[**1342].Hide Hea

In [24]:
pd.options.display.max_columns = 200

In [28]:
merged[merged['caseId']=='2010-001']

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,firstAgreement,secondAgreement
73859,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-01,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,111,JGRoberts,1.0,1.0,1.0,2.0,NaN,NaN
73860,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-02,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,105,AScalia,1.0,1.0,1.0,2.0,NaN,NaN
73861,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-03,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,106,AMKennedy,1.0,1.0,1.0,2.0,NaN,NaN
73862,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-04,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,108,CThomas,1.0,1.0,1.0,2.0,NaN,NaN
73863,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-05,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,109,RBGinsburg,1.0,1.0,1.0,2.0,NaN,NaN
73864,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-06,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,110,SGBreyer,1.0,1.0,1.0,2.0,NaN,NaN
73865,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-07,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,112,SAAlito,1.0,1.0,1.0,2.0,NaN,NaN
73866,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-08,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 8663,2010,1704,Roberts,10-91,"BILL K. WILSON, SUPERINTENDENT, INDIANA STATE ...",NaN,NaN,28,18.0,215,NaN,1,NaN,NaN,0.0,68.0,NaN,27.0,NaN,0.0,12.0,6.0,2.0,1,5.0,0,1,0,0,10020.0,1.0,1.0,0.0,4.0,NaN,3.0,341.0,NaN,NaN,111.0,1,9,0,113,SSotomayor,1.0,1.0,1.0,2.0,NaN,NaN
73867,2010-001,2010-001-01,2010-001-01-01,2010-001-01-01-01-09,11/8/2010,2,NaN,131 S. Ct. 13,178 L. Ed. 2d 276,2010 U.S. LEXIS 

In [ ]:
merged.head()

In [18]:
merged_scotus_judges = pd.read_csv('/Volumes/Seagate Backup Plus Drive/Dropbox (Daniel Chen)/Oyez_vowels/merged_scotus_judges.csv')

In [19]:
predictions = pd.read_csv('/Volumes/Seagate Backup Plus Drive/Dropbox (Daniel Chen)/Oyez_vowels/predictions.csv')


In [20]:
vowels = pd.read_csv('/Volumes/Seagate Backup Plus Drive/Dropbox (Daniel Chen)/Oyez_vowels/vowels.csv')

In [38]:
merged_scotus_judges.head()

,SpeakerId,docket_id,matched_name,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,term,naturalCourt,chief,docket,caseName,dateArgument,dateRearg,petitioner,petitionerState,respondent,respondentState,jurisdiction,adminAction,adminActionState,threeJudgeFdc,caseOrigin,caseOriginState,caseSource,caseSourceState,lcDisagreement,certReason,lcDisposition,lcDispositionDirection,declarationUncon,caseDisposition,caseDispositionUnusual,partyWinning,precedentAlteration,voteUnclear,issue,issueArea,decisionDirection,decisionDirectionDissent,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,firstAgreement,secondAgreement
0,1.0,1998_96_1570,David H. Souter,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-06,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,107,DHSouter,1.0,1.0,1.0,2.0,NaN,NaN
1,2.0,1998_96_1570,Antonin Scalia,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-04,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,105,AScalia,1.0,1.0,1.0,2.0,NaN,NaN
2,5.0,1998_96_1570,John Paul Stevens,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-02,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,103,JPStevens,1.0,1.0,1.0,2.0,NaN,NaN
3,6.0,1998_96_1570,Anthony M. Kennedy,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-05,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,106,AMKennedy,1.0,1.0,1.0,2.0,NaN,NaN
4,8.0,1998_96_1570,Ruth Bader Ginsburg,1998-008,1998-008-01,1998-008-01-01,1998-008-01-01-01-08,12/14/1998,1,525 U.S. 128,119 S. Ct. 493,142 L. Ed. 2d 510,1998 U.S. LEXIS 8080,1998,1607,Rehnquist,96-1570,"NYNEX CORPORATION, et al. v. DISCON, INCORPORATED",10/5/1998,NaN,122.0,NaN,233.0,NaN,1,NaN,NaN,0,NaN,NaN,22.0,NaN,0,7.0,2.0,2.0,1.0,5.0,0.0,1.0,0.0,0.0,80010.0,8.0,1.0,0,4.0,NaN,3.0,367.0,NaN,110.0,102.0,1,9,0,109,RBGinsburg,1.0,1.0,1.0,2.0,NaN,NaN


In [39]:
predictions.head()

,docketId,docket_id,justice,SpeakerId,admin_action,case_origin,case_origin_circuit,case_source,case_source_circuit,cert_reason,court_direction_circuit_origin_mean,court_direction_circuit_origin_mean_10,court_direction_circuit_source_mean,court_direction_circuit_source_mean_10,court_direction_issue_mean,court_direction_issue_mean_10,court_direction_mean,court_direction_mean_10,court_direction_petitioner_mean,court_direction_petitioner_mean_10,court_direction_respondent_mean,court_direction_respondent_mean_10,current_court_agree_mean,current_court_agree_std,current_court_direction_circuit_origin_mean,current_court_direction_circuit_origin_std,current_court_direction_circuit_source_mean,current_court_direction_circuit_source_std,current_court_direction_issue_mean,current_court_direction_issue_mean_z,current_court_direction_issue_std,current_court_direction_mean,current_court_direction_petitioner_mean,current_court_direction_petitioner_std,current_court_direction_respondent_mean,current_court_direction_respondent_std,current_court_direction_std,diff_court_lc_direction,diff_court_lc_direction_abs,diff_court_lc_direction_abs_z,diff_court_lc_direction_z,diff_justice_court_direction,diff_justice_court_direction_abs,diff_justice_court_direction_issue,diff_justice_court_direction_issue_abs,diff_justice_court_direction_issue_z,diff_justice_court_direction_petitioner,diff_justice_court_direction_petitioner_abs,diff_justice_court_direction_respondent,diff_justice_court_direction_respondent_abs,...,issue,issue_area,jurisdiction,justice_agree_mean,justice_agree_mean_10,justice_direction_circuit_origin_mean,justice_direction_circuit_origin_mean_10,justice_direction_circuit_source_mean,justice_direction_circuit_source_mean_10,justice_direction_issue_mean,justice_direction_issue_mean_10,justice_direction_issue_mean_z,justice_direction_mean,justice_direction_mean_10,justice_direction_mean_z,justice_direction_petitioner_mean,justice_direction_petitioner_mean_10,justice_direction_respondent_mean,justice_direction_respondent_mean_10,law_type,lc_direction,lc_disagreement,lc_disposition,lower_court_direction_circuit_source_mean,lower_court_direction_circuit_source_mean_10,lower_court_direction_issue_mean,lower_court_direction_issue_mean_10,lower_court_direction_petitioner_mean,lower_court_direction_petitioner_mean_10,lower_court_direction_respondent_mean,lower_court_direction_respondent_mean_10,month_argument,month_decision,natural_court,outcome,party_president,petitioner,petitioner_dk,respondent,respondent_dk,segal_cover,year_of_birth,prediction,prob_prediction0,prob_prediction1,target,docket_vote_mean,docket_vote_sum,correct,year
0,1998-008-01,1998_96_1570,107,1.0,-99.0,-99.0,-99.0,22.0,2.0,7.0,-99.0,-99.0,1.507752,1.1,1.680693,1.5,1.521238,1.7,1.526316,1.4,1.647059,1.7,0.802987,0.044931,-99.0,-99.0,1.411376,0.111761,1.452491,0.176781,0.147878,1.435138,1.22619,0.178174,1.457937,0.206663,0.110706,-0.564862,0.564862,0.906323,-98.0,0.085207,0.085207,0.047509,0.047509,0.321275,0.107143,0.107143,0.042063,0.042063,...,80010.0,8.0,1,0.830588,0.9,-99.0,-99.0,1.487179,1.2,1.500000,1.5,0.357812,1.520345,1.8,0.074195,1.333333,-99.0,1.500000,-99.0,3.0,2.0,0,2.0,1.47907,1.7,1.371287,1.6,1.631579,1.6,1.235294,1.1,10.0,12,1607,1,1,122.0,1.0,233.0,1.0,0.325,1939,0,0.662674,0.337326,1,0,0,False,1998
1,1998-008-01,1998_96_1570,105,2.0,-99.0,-99.0,-99.0,22.0,2.0,7.0,-99.0,-99.0,1.507752,1.1,1.680693,1.5,1.521238,1.7,1.526316,1.4,1.647059,1.7,0.802987,0.044931,-99.0,-99.0,1.411376,0.111761,1.452491,0.176821,0.147878,1.435138,1.22619,0.178174,1.457937,0.206663,0.110706,-0.564862,0.564862,0.906689,-98.0,-0.083537,0.083537,-0.008046,0.008046,-0.054411,-0.083333,0.083333,0.042063,0.042063,...,80010.0,8.0,1,0.820178,0.8,-99.0,-99.0,1.421053,-99.0,1.444444,1.3,0.356150,1.351601,1.5,-0.038430,1.142857,-99.0,1.500000,-99.0,3.0,2.0,0,2.0,1.47907,1.7,1.371287,1.6,1.631579,1.6,1.235294,1.1,10.0,12,1607,1,1,122.0,1.0,233.0,1.0,0.000,1936,0,0.633697,0.366303,1,0,0,False,1998
2,199

In [42]:
vowels

,stress,vowel,vowel_id
0,0.0,AW,0
1,1.0,AY,1
2,0.0,AA,2
3,1.0,OY,3
4,2.0,ER,4
5,2.0,UH,5
6,2.0,EY,6
7,1.0,AE,7
8,0.0,IY,8
9,0.0,EH,9
